# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
indicator_list = pattern
indicator_list_higher = pattern

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 1000
load = False

print(f'Timeframe is {higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = pattern
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'Pattern': {'use_vol': [-1], 'window_low_bound': [1], 'window_high_bound': [6],
                            'first_candle': [0.7], 'second_candle': [0.5], 
                            'third_candle': [0.5]},
                'LinearReg': {'timeperiod': [8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

# stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
# if not stat_list:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
# else:
#     stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')


Timeframe is 1h, trade type is buy
Number of combinations is 3


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [03:48<00:00, 76.15s/it]


In [3]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,8,0,1200,1.568763,0.535000,682.515000,642.000000,0.9607,0.9956,0.9999,1.0684,1.5281,1.5591,1.5875,1.5984,1.5770,1.6034,1.6403,1.6478,1.6599,1.6646,1.6940,1.7131,1.7315,1.7504,1.7258,1.7387,1.7684,1.8084,1.8118,1.8175,0.35,0.31,0.33,0.37,0.43,0.45,0.43,0.43,0.44,0.46,0.45,0.52,0.50,0.53,0.58,0.63,0.65,0.66,0.65,0.68,0.76,0.77,0.78,0.68
1,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,10,0,1240,1.548383,0.544583,679.995333,675.283333,0.9382,0.9747,0.9738,1.0425,1.4891,1.5237,1.5575,1.5746,1.5514,1.5750,1.6111,1.6158,1.6334,1.6359,1.6687,1.6923,1.7126,1.7335,1.7135,1.7348,1.7662,1.8068,1.8112,1.8249,0.36,0.32,0.33,0.36,0.44,0.45,0.43,0.42,0.43,0.44,0.45,0.51,0.50,0.54,0.57,0.66,0.66,0.67,0.65,0.72,0.80,0.79,0.82,0.75
2,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,12,0,1288,1.545679,0.535833,702.834767,690.153333,0.9366,0.9678,0.9832,1.0593,1.4853,1.5176,1.5512,1.5626,1.5394,1.5649,1.5989,1.6051,1.6221,1.6295,1.6644,1.6907,1.7163,1.7353,1.7123,1.7346,1.7671,1.8099,1.8139,1.8283,0.35,0.32,0.34,0.37,0.43,0.44,0.41,0.40,0.43,0.42,0.44,0.49,0.48,0.52,0.57,0.63,0.65,0.67,0.66,0.72,0.80,0.78,0.80,0.74


In [8]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
0,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,8,0,1200,1.568763,0.535000,682.515000,642.000000,0.9607,0.9956,0.9999,1.0684,1.5281,1.5591,1.5875,1.5984,1.5770,1.6034,1.6403,1.6478,1.6599,1.6646,1.6940,1.7131,1.7315,1.7504,1.7258,1.7387,1.7684,1.8084,1.8118,1.8175,0.35,0.31,0.33,0.37,0.43,0.45,0.43,0.43,0.44,0.46,0.45,0.52,0.50,0.53,0.58,0.63,0.65,0.66,0.65,0.68,0.76,0.77,0.78,0.68
1,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,10,0,1240,1.548383,0.544583,679.995333,675.283333,0.9382,0.9747,0.9738,1.0425,1.4891,1.5237,1.5575,1.5746,1.5514,1.5750,1.6111,1.6158,1.6334,1.6359,1.6687,1.6923,1.7126,1.7335,1.7135,1.7348,1.7662,1.8068,1.8112,1.8249,0.36,0.32,0.33,0.36,0.44,0.45,0.43,0.42,0.43,0.44,0.45,0.51,0.50,0.54,0.57,0.66,0.66,0.67,0.65,0.72,0.80,0.79,0.82,0.75
2,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,12,0,1288,1.545679,0.535833,702.834767,690.153333,0.9366,0.9678,0.9832,1.0593,1.4853,1.5176,1.5512,1.5626,1.5394,1.5649,1.5989,1.6051,1.6221,1.6295,1.6644,1.6907,1.7163,1.7353,1.7123,1.7346,1.7671,1.8099,1.8139,1.8283,0.35,0.32,0.34,0.37,0.43,0.44,0.41,0.40,0.43,0.42,0.44,0.49,0.48,0.52,0.57,0.63,0.65,0.67,0.66,0.72,0.80,0.78,0.80,0.74
3,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,14,0,1339,1.540700,0.523750,723.997300,701.301250,0.9386,0.9691,0.9827,1.0594,1.4677,1.5002,1.5414,1.5535,1.5350,1.5639,1.5952,1.6022,1.6161,1.6227,1.6577,1.6846,1.7100,1.7317,1.7102,1.7311,1.7613,1.8045,1.8093,1.8287,0.35,0.32,0.33,0.36,0.41,0.43,0.40,0.39,0.41,0.42,0.44,0.49,0.46,0.52,0.55,0.62,0.64,0.65,0.63,0.69,0.76,0.77,0.79,0.74
4,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,16,0,1364,1.535954,0.519583,731.041483,708.711667,0.9484,0.9755,0.9869,1.0588,1.4562,1.4897,1.5280,1.5420,1.5239,1.5514,1.5836,1.5923,1.6073,1.6166,1.6517,1.6786,1.7039,1.7264,1.7051,1.7330,1.7616,1.8042,1.8086,1.8292,0.35,0.32,0.33,0.36,0.41,0.43,0.41,0.39,0.41,0.43,0.43,0.49,0.46,0.49,0.54,0.61,0.62,0.65,0.62,0.69,0.76,0.77,0.78,0.72
5,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,18,0,1398,1.510254,0.505417,713.335325,706.572500,0.9455,0.9674,0.9812,1.0495,1.4357,1.4655,1.5011,1.5047,1.4899,1.5170,1.5520,1.5595,1.5747,1.5864,1.6216,1.6500,1.6728,1.6952,1.6753,1.7052,1.7349,1.7768,1.7816,1.8026,0.35,0.32,0.33,0.36,0.40,0.42,0.40,0.38,0.40,0.41,0.42,0.47,0.44,0.48,0.53,0.60,0.59,0.63,0.60,0.66,0.75,0.76,0.76,0.67
6,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,20,0,1422,1.501158,0.502083,712.647150,713.962500,0.9467,0.9714,0.9783,1.0428,1.4211,1.4542,1.4895,1.4969,1.4853,1.5099,1.5450,1.5518,1.5659,1.5710,1.6042,1.6350,1.6583,1.6806,1.6624,1.6951,1.7268,1.7683,1.7732,1.7941,0.35,0.32,0.33,0.34,0.39,0.41,0.39,0.38,0.41,0.41,0.42,0.46,0.44,0.48,0.53,0.59,0.58,0.62,0.59,0.66,0.75,0.76,0.77,0.67


# Check local statistics

In [2]:
# 15m/1h
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']

pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('e_ratio_avg', ascending=False)
stat.head(20)

,pattern,Pattern_use_vol,Pattern_window_low_bound,Pattern_window_high_bound,Pattern_first_candle,Pattern_second_candle,Pattern_third_candle,LinearReg_timeperiod,LinearReg_low_bound,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
71,Pattern_LinearReg,-1,1,6,0.8,0.7,0.5,8,0,695,1.715071,0.520000,496.974229,361.400000,1.0074,1.1020,1.0940,1.1651,1.8522,1.8394,1.8354,1.7891,1.7376,1.7629,1.8117,1.8146,1.7924,1.7851,1.7947,1.8131,1.8444,1.8807,1.8626,1.8683,1.9016,1.9321,1.9354,1.9399,0.32,0.31,0.33,0.34,0.49,0.43,0.42,0.38,0.37,0.45,0.47,0.52,0.46,0.49,0.51,0.63,0.65,0.70,0.70,0.67,0.76,0.77,0.72,0.59
67,Pattern_LinearReg,-1,1,6,0.8,0.6,0.5,8,0,895,1.675829,0.567083,604.867104,507.539583,1.0086,1.0568,1.0507,1.1042,1.6602,1.6673,1.6852,1.6874,1.6723,1.7105,1.7613,1.7658,1.7611,1.7788,1.8053,1.8197,1.8473,1.8817,1.8656,1.8768,1.8999,1.9427,1.9559,1.9548,0.33,0.32,0.32,0.35,0.44,0.40,0.42,0.41,0.41,0.50,0.47,0.55,0.49,0.56,0.62,0.75,0.73,0.76,0.77,0.76,0.86,0.88,0.78,0.73
63,Pattern_LinearReg,-1,1,6,0.8,0.5,0.5,8,0,1026,1.634500,0.573750,650.997000,588.667500,1.0313,1.0635,1.0633,1.1199,1.6290,1.6354,1.6569,1.6607,1.6369,1.6736,1.7152,1.7124,1.7109,1.7291,1.7530,1.7659,1.7883,1.8112,1.7933,1.8082,1.8337,1.8669,1.8811,1.8883,0.34,0.33,0.34,0.37,0.46,0.43,0.43,0.46,0.42,0.49,0.48,0.56,0.51,0.56,0.63,0.75,0.72,0.76,0.75,0.76,0.87,0.85,0.78,0.72
51,Pattern_LinearReg,-1,1,6,0.7,0.7,0.5,8,0,914,1.632650,0.514583,578.242100,470.329167,0.9698,1.0469,1.0562,1.1248,1.6911,1.7113,1.7200,1.6929,1.6479,1.6626,1.7029,1.7109,1.7179,1.6951,1.7108,1.7340,1.7604,1.8002,1.7907,1.7941,1.8302,1.8682,1.8698,1.8749,0.34,0.32,0.32,0.34,0.45,0.45,0.46,0.39,0.41,0.46,0.46,0.54,0.49,0.53,0.51,0.62,0.57,0.65,0.66,0.64,0.72,0.68,0.72,0.62
47,Pattern_LinearReg,-1,1,6,0.7,0.6,0.5,8,0,1176,1.584392,0.539167,687.244600,634.060000,0.9901,1.0255,1.0249,1.0720,1.5252,1.5504,1.5737,1.5872,1.5718,1.5996,1.6402,1.6516,1.6692,1.6700,1.7040,1.7214,1.7409,1.7748,1.7668,1.7779,1.8059,1.8527,1.8640,1.8656,0.35,0.32,0.31,0.34,0.41,0.43,0.43,0.40,0.45,0.46,0.46,0.51,0.49,0.55,0.60,0.67,0.66,0.68,0.69,0.69,0.77,0.78,0.77,0.72
43,Pattern_LinearReg,-1,1,6,0.7,0.5,0.5,8,0,1357,1.572608,0.562500,777.029508,763.312500,1.0107,1.0404,1.0465,1.1024,1.5180,1.5438,1.5723,1.5847,1.5647,1.5896,1.6283,1.6394,1.6572,1.6575,1.6895,1.7049,1.7247,1.7499,1.7348,1.7472,1.7751,1.8117,1.8210,1.8283,0.36,0.33,0.34,0.38,0.45,0.45,0.46,0.45,0.47,0.49,0.47,0.54,0.51,0.58,0.62,0.70,0.67,0.71,0.70,0.71,0.80,0.78,0.78,0.75
31,Pattern_LinearReg,-1,1,6,0.6,0.7,0.5,8,0,1180,1.566383,0.520000,668.332333,613.600000,0.9510,0.9895,1.0199,1.1030,1.5656,1.5732,1.6006,1.6157,1.5738,1.5883,1.6316,1.6449,1.6506,1.6249,1.6493,1.6677,1.6995,1.7389,1.7429,1.7395,1.7749,1.8053,1.8167,1.8259,0.35,0.33,0.32,0.33,0.45,0.45,0.47,0.44,0.45,0.45,0.44,0.47,0.47,0.51,0.51,0.64,0.65,0.67,0.64,0.64,0.70,0.68,0.72,0.70
11,Pattern_LinearReg,-1,1,6,0.5,0.7,0.5,8,0,1377,1.563804,0.516250,776.358337,710.876250,0.9451,0.9839,1.0166,1.0888,1.4895,1.5277,1.5636,1.5828,1.5696,1.5840,1.6219,1.6434,1.6477,1.6316,1.6600,1.6903,1.7174,1.7549,1.7568,1.7518,1.7931,1.8213,1.8403,1.8492,0.34,0.33,0.32,0.34,0.45,0.43,0.42,0.41,0.45,0.44,0.41,0.47,0.47,0.53,0.53,0.63,0.65,0.67,0.67,0.67,0.70,0.69,0.72,0.65
171,Pattern_LinearReg,0,1,6,0.8,0.7,0.5,8,0,1149,1.534212,0

# Research parameter influence

In [9]:
param = 'Pattern_use_vol'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Pattern_use_vol,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,-1,1.374522,0.456817,341.774870,413.046021,88818
1,0,1.330304,0.445579,486.238149,636.135771,141278
2,1,1.177491,0.232975,51.898347,63.730733,23904


# Plot result in file

In [23]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = param, 'e_ratio_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig(f"{'_'.join(pattern)}'_'{ttype}_{param}", dpi=fig.dpi)

# Save new config data to config file

In [11]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['Pattern', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'Pattern': {'window_low_bound': [1], 'window_high_bound': [6],
                            'first_candle': [0.5], 'second_candle': [0.5]},
                'LinearReg': {'timeperiod': [3], 'low_bound': [0]}
             }

        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)